1. Install Required Packages (Code Cell)

In [1]:
# Install amplpy and python-dotenv
%pip install -q amplpy python-dotenv


[notice] A new release of pip is available: 23.0.1 -> 25.0
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


2. Environment Setup (Markdown Explanation)
Create a .env file in your working directory with:
Replace your_license_uid_here with your actual AMPL license UUID.

3. Initialize AMPL with Environment Variables (Code Cell)

In [2]:
from amplpy import AMPL, ampl_notebook
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Initialize AMPL with security best practices
ampl = ampl_notebook(
    modules=["highs"],
    license_uuid=os.getenv("AMPL_LICENSE_UUID")  # Secure credential handling
)
print("AMPL initialized successfully ✅")

Licensed to AMPL Community Edition License for <laleso2563@perceint.com>.
AMPL initialized successfully ✅


4. Define the Optimization Model (Code Cell with Markdown Explanation)


In [6]:
%%ampl_eval
reset;
# --------------------------
# Knapsack Problem Formulation
# --------------------------
set ITEMS;                # All available items
param weight{ITEMS};      # Weight of each item
param benefit{ITEMS};     # Benefit value of each item

var H{ITEMS} binary;      # Husband's knapsack (1 = packed)
var W{ITEMS} binary;      # Wife's knapsack (1 = packed)

maximize TotalBenefit:
    sum{i in ITEMS} benefit[i] * (H[i] + W[i]);

subject to HusbandCapacity:
    sum{i in ITEMS} weight[i] * H[i] <= 20;

subject to WifeCapacity:
    sum{i in ITEMS} weight[i] * W[i] <= 17;

subject to NoDuplicates{i in ITEMS}:
    H[i] + W[i] <= 1;     # Item cannot be in both knapsacks

5. Input Data Handling (Code Cell with Markdown Explanation)


In [7]:
# --------------------------
# Dynamic Data Input System
# --------------------------
items = {
    'stove': {'weight': 7, 'benefit': 10},
    'lamp': {'weight': 6, 'benefit': 9},
    'axe': {'weight': 13, 'benefit': 6},
    'binoculars': {'weight': 4, 'benefit': 3},
    'rope': {'weight': 9, 'benefit': 14}
}

# Send data to AMPL
ampl.set["ITEMS"] = list(items.keys())
ampl.param["weight"] = {k: v['weight'] for k, v in items.items()}
ampl.param["benefit"] = {k: v['benefit'] for k, v in items.items()}

print("Data loaded successfully 📊")

Data loaded successfully 📊


6. Solve the Problem (Code Cell with Markdown Explanation)


In [8]:
# --------------------------
# Solver Configuration
# --------------------------
ampl.option["solver"] = "highs"  # Open-source solver
ampl.solve()

# Check solution status
if ampl.solve_result != "solved":
    raise RuntimeError(f"Solver failed: {ampl.solve_message}")

HiGHS 1.8.1: HiGHS 1.8.1: optimal solution; objective 39
16 simplex iterations
1 branching nodes


7. Display Results (Code Cell with Markdown Explanation)


In [9]:
# --------------------------
# Professional Results Display
# --------------------------
import pandas as pd

# Get results
results = []
for item in ampl.getSet("ITEMS"):
    h_val = ampl.getVariable("H").get(item).value()
    w_val = ampl.getVariable("W").get(item).value()
    results.append({
        'Item': item,
        'Husband': '✅' if h_val > 0.5 else '❌',
        'Wife': '✅' if w_val > 0.5 else '❌',
        'Weight': items[item]['weight'],
        'Benefit': items[item]['benefit']
    })

# Create DataFrame
df = pd.DataFrame(results).set_index('Item')

# Format display
print(f"\nOptimal Total Benefit: {ampl.getObjective('TotalBenefit').value():.1f}")
print("\nAllocation Matrix:")
display(df.style.format({
    'Weight': '{:.0f} kg',
    'Benefit': '{:.0f} pts'
}).set_caption("Knapsack Optimization Results"))


Optimal Total Benefit: 39.0

Allocation Matrix:


,Husband,Wife,Weight,Benefit
Item,,,,
stove,❌,✅,7 kg,10 pts
lamp,✅,❌,6 kg,9 pts
axe,✅,❌,13 kg,6 pts
binoculars,❌,❌,4 kg,3 pts
rope,❌,✅,9 kg,14 pts
